# Understanding the data we are given

In [49]:
import pickle
import numpy as np

# Load the variables
with open('nonnested_cval_mltsatt_l819_first.pkl', 'rb') as f:
    results = pickle.load(f)

## Results Analyse

In [50]:
type(results),len(results),results.keys()

(dict,
 14,
 dict_keys(['x', 'y', 'demographics', 'optim_scoring', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [51]:
results["demographics"].keys()

dict_keys(['gender', 'year', 'language', 'label', 'stratifier_col', 'oversampler_col'])

In [52]:
results["optim_scoring"], type(results["optim_scoring"])

('roc', str)

In [53]:
print(results[1].keys())

dict_keys(['train_index', 'test_index', 'oversample_indexes', 'x_resampled', 'y_resampled', 'x_resampled_train', 'y_resampled_train', 'x_resampled_val', 'y_resampled_val', 'best_params', 'gender', 'year', 'language', 'label', 'y_pred', 'y_proba', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'roc', 'tp', 'fp', 'fn'])


In [54]:
results[1]["x_resampled_train"][0][0], results[1]["x_resampled_train"][-1][0]

([0, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 5.447])

In [56]:
# Make mean of the results acrross the 10 folds
mean_roc = []
for i in range(10):
    mean_roc.append(results[i]["roc"])
mean_roc = np.array(mean_roc).mean()

print("Mean ROC: ", mean_roc)

Mean ROC:  0.6283116883116884


## Full data, sequences and labels

In [ ]:
with open('sequences.pkl', 'rb') as f:
    sequences = pickle.load(f)

with open('labels.pkl', 'rb') as f:
    labels = pickle.load(f)

with open('demographics.pkl', 'rb') as f:
    demographics = pickle.load(f)

with open('full_data.pkl', 'rb') as f:
    full_data = pickle.load(f)

In [2]:
student_index = 0
full_data[student_index].keys()

dict_keys(['path', 'length', 'learner_id', 'raw_sequence', 'raw_begin', 'raw_end', 'last_timestamp', 'permutation', 'gender', 'year', 'language', 'begin', 'end', 'break_sequence', 'sequence', 'label', '1hot-sequence', 'nobreak', 'nobreak-1hot-sequence', 'stratifier_column'])

In [3]:
print("normal sequence", full_data[student_index]['sequence'][0:2])
print("1-hot encoded",full_data[student_index]['1hot-sequence'][0:2])
print("break_sequence", full_data[student_index]['break_sequence'][0:2])

normal sequence [[0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3440621047581669], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0]]
1-hot encoded [[0, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]]
break_sequence [[0, 0, 0, 1, 0, 0, 0, 0, 0, 7.889], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 75.141, 0.0]]


In [4]:
type(sequences), len(sequences) ,sequences[0][0:3] # number student and senquences

(list,
 254,
 [[0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]])

In [11]:
# See the max and min length of the sequences
max_len = 0
min_len = 1000
for seq in sequences:
    if len(seq) > max_len:
        max_len = len(seq)
    if len(seq) < min_len:
        min_len = len(seq)
print(max_len, min_len)

len(sequences[10])

819 13


102

In [5]:
len(labels) , labels[0:7]

(254, [1, 1, 1, 1, 1, 1, 0])

In [6]:
type(demographics), len(demographics) , demographics.keys()

(dict,
 6,
 dict_keys(['gender', 'year', 'language', 'label', 'stratifier_col', 'oversampler_col']))

In [7]:
# line 83 form nonested_cv.py used in ml_pipline.py
"""demographics_train = {}
for demo in demographics:
    demographics_train[demo] = [
        demographics[demo][idx] for idx in train_index
    ]"""

for demo in demographics:
    print(demo)

gender
year
language
label
stratifier_col
oversampler_col


In [8]:
len(demographics["language"]), demographics["language"][7:16]

(254,
 ['Français',
  'Deutsch',
  'Deutsch',
  'Deutsch',
  'Deutsch',
  'Deutsch',
  'Français',
  'Français',
  'Français'])

In [9]:
len(demographics["oversampler_col"]), demographics["oversampler_col"][7:16]

(254,
 ['0Français',
  '0Deutsch',
  '1Deutsch',
  '1Deutsch',
  '0Deutsch',
  '0Deutsch',
  '0Français',
  '1Français',
  '0Français'])